In [2]:
import requests

url = 'https://newsapi.org/v2/everything'
params = {
    'q': 'viral hemorrhagic fever',  # Search for specific keywords
    'from': '2023-01-01',            # Start date for articles
    'sortBy': 'publishedAt',
    'apiKey': '9c40dfed39574e099b7f036dcbf30ed8'         # Your API key here
}

response = requests.get(url, params=params)
articles = response.json()['articles']

for article in articles:
    print(article['title'], article['description'], article['url'])


KeyError: 'articles'

In [3]:
import requests

url = 'https://newsapi.org/v2/everything'
params = {
    'q': 'viral hemorrhagic fever',  # Search for specific keywords
    'from': '2023-01-01',            # Start date for articles
    'sortBy': 'publishedAt',
    'apiKey': '9c40dfed39574e099b7f036dcbf30ed8'         # Your API key here
}

response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    response_data = response.json()
    if 'articles' in response_data:
        articles = response_data['articles']
        for article in articles:
            print(article['title'], article['description'], article['url'])
    else:
        print("No articles found. Response: ", response_data)
else:
    print("Failed request. Status code:", response.status_code)
    print("Response:", response.text)


Failed request. Status code: 426
Response: {"status":"error","code":"parameterInvalid","message":"You are trying to request results too far in the past. Your plan permits you to request articles as far back as 2024-08-29, but you have requested 2023-01-01. You may need to upgrade to a paid plan."}


In [4]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.bbc.com/news'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

articles = soup.find_all('h3', class_='gs-c-promo-heading__title')

for article in articles:
    print(article.text)


In [5]:
for article in articles:
    title = article.text
    link = article.find('a')['href']  # Extract the URL of the article
    full_link = f"https://www.bbc.com{link}"  # Ensure it's a full URL

    print(f"Title: {title}")
    print(f"URL: {full_link}")
    print("-------------------------")


In [6]:
import csv

# Open a CSV file for writing
with open('bbc_articles.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'URL', 'Content'])  # Write header row

    # Loop through articles and write the details into the CSV
    for article in articles:
        title = article.text
        link = article.find('a')['href']
        full_link = f"https://www.bbc.com{link}"

        # Request the article page
        article_page = requests.get(full_link)
        article_soup = BeautifulSoup(article_page.text, 'html.parser')

        # Extract article content
        paragraphs = article_soup.find_all('p')
        article_content = ' '.join([para.text for para in paragraphs])

        # Write the data to the CSV file
        writer.writerow([title, full_link, article_content])


In [ ]:
import praw

# Reddit API credentials
reddit = praw.Reddit(client_id='YOUR_CLIENT_ID',
                     client_secret='YOUR_CLIENT_SECRET',
                     user_agent='YOUR_USER_AGENT')

# Scraping posts from a subreddit (e.g., r/epidemiology)
subreddit = reddit.subreddit('epidemiology')

# Search for specific keywords in titles
for post in subreddit.search('viral hemorrhagic fever', limit=100):
    print(post.title, post.selftext)

In [1]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.who.int/news-room/articles'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

articles = soup.find_all('h2', class_='article__title')

for article in articles:
    print(article.text, article.find('a')['href'])


In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to get Google News search results
def scrape_google_news(query, from_date, to_date):
    url = f'https://www.google.com/search?q={query}+after:{from_date}+before:{to_date}&tbm=nws'
    headers = {'User-Agent': 'Mozilla/5.0'}

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    articles = []

    for item in soup.find_all('div', class_='dbsr'):
        title = item.find('div', class_='JheGif').get_text()
        link = item.a['href']
        snippet = item.find('div', class_='Y3v8qd').get_text()

        articles.append({
            'title': title,
            'link': link,
            'snippet': snippet
        })

    return articles

# Define query and date range
search_query = 'viral hemorrhagic fever OR Lassa fever OR Ebola'
from_date = '2017-06-01'
to_date = '2023-06-30'

# Scrape Google News
articles = scrape_google_news(query=search_query, from_date=from_date, to_date=to_date)

# Save results to a DataFrame and export as CSV
df = pd.DataFrame(articles)
df.to_csv('google_news_articles.csv', index=False)

print(f"Scraped {len(df)} articles from Google News.")


Scraped 0 articles from Google News.


In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to get Google News search results
def scrape_google_news(query, from_date, to_date):
    # Modify the search query to include the date range
    url = f'https://www.google.com/search?q={query}+after:{from_date}+before:{to_date}&tbm=nws'
    
    # Set user-agent to avoid being blocked
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    # Send the request to Google
    response = requests.get(url, headers=headers)
    
    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract articles from the HTML
    articles = []
    
    for result in soup.select('.dbsr'):
        try:
            title = result.select_one('.nDgy9d').get_text()  # Update with correct class
            link = result.find('a')['href']
            snippet = result.select_one('.Y3v8qd').get_text()  # Update with correct class
            
            articles.append({
                'title': title,
                'link': link,
                'snippet': snippet
            })
        except AttributeError as e:
            # Handle the case where the article doesn't have the expected structure
            print(f"Skipping an article due to missing data: {e}")
    
    return articles

# Define query and date range
search_query = 'viral hemorrhagic fever OR Lassa fever OR Ebola'
from_date = '2017-06-01'
to_date = '2023-06-30'

# Scrape Google News
articles = scrape_google_news(query=search_query, from_date=from_date, to_date=to_date)

# Save results to a DataFrame and export as CSV
if articles:
    df = pd.DataFrame(articles)
    df.to_csv('google_news_articles.csv', index=False)
    print(f"Scraped {len(df)} articles from Google News.")
else:
    print("No articles found.")


No articles found.


In [10]:
pip install feedparser

  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=561a83422bfeabbd2262e57730afa297f2b2b8385b58cb6584122a08a2e88001
  Stored in directory: /Users/m1/Library/Caches/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k
Note: you may need to restart the kernel to use updated packages.


In [2]:
import feedparser
import urllib.parse
import pandas as pd

# Function to fetch Google News RSS feed
def fetch_google_news_rss(query):
    # URL-encode the search query
    encoded_query = urllib.parse.quote(query)
    
    # Modify the RSS URL to use the encoded query
    rss_url = f"https://news.google.com/rss/search?q={encoded_query}&hl=en-US&gl=US&ceid=US:en"
    
    # Parse the RSS feed
    feed = feedparser.parse(rss_url)
    
    articles = []
    
    for entry in feed.entries:
        articles.append({
            'title': entry.title,
            'link': entry.link,
            'published': entry.published,
            'summary': entry.summary
        })
    
    return articles

# Define query (search for keywords related to your topic)
search_query = 'Lassa fever OR Ebola OR viral hemorrhagic fever'

# Fetch articles
articles = fetch_google_news_rss(query=search_query)

# Check if we have articles
if articles:
    # Save to a DataFrame and export as CSV
    df = pd.DataFrame(articles)
    df.to_csv('google_news_rss_articles.csv', index=False)
    print(f"Fetched {len(df)} articles from Google News RSS.")
else:
    print("No articles found.")


Fetched 98 articles from Google News RSS.


In [3]:
import feedparser
import urllib.parse
import pandas as pd
from datetime import datetime

# Function to fetch Google News RSS feed
def fetch_google_news_rss(query):
    # URL-encode the search query
    encoded_query = urllib.parse.quote(query)
    
    # Modify the RSS URL to use the encoded query
    rss_url = f"https://news.google.com/rss/search?q={encoded_query}&hl=en-US&gl=US&ceid=US:en"
    
    # Parse the RSS feed
    feed = feedparser.parse(rss_url)
    
    articles = []
    
    # Define the date range for filtering articles (from January 2017 to June 2023)
    start_date = datetime(2017, 1, 1)
    end_date = datetime(2023, 6, 30)
    
    for entry in feed.entries:
        # Convert the published date to a datetime object
        published_date = datetime.strptime(entry.published, '%a, %d %b %Y %H:%M:%S %Z')
        
        # Check if the article is within the specified date range
        if start_date <= published_date <= end_date:
            # Extract the publisher/source (if available)
            publisher = entry.get('source', {}).get('title', 'Unknown Publisher')

            # Append relevant details to the articles list
            articles.append({
                'title': entry.title,
                'link': entry.link,
                'published': published_date,
                'publisher': publisher,
                'summary': entry.summary
            })
    
    return articles

# Define query (search for keywords related to your topic)
search_query = 'Lassa fever OR Ebola OR viral hemorrhagic fever'

# Fetch articles
articles = fetch_google_news_rss(query=search_query)

# Check if we have articles
if articles:
    # Save to a DataFrame and export as CSV
    df = pd.DataFrame(articles)
    df.to_csv('google_news_rss_articles_filtered.csv', index=False)
    print(f"Fetched {len(df)} articles from Google News RSS.")
else:
    print("No articles found.")


Fetched 35 articles from Google News RSS.


In [5]:
import feedparser
import urllib.parse
import pandas as pd
from datetime import datetime

# Function to fetch Google News RSS feed
def fetch_google_news_rss(query):
    # URL-encode the search query
    encoded_query = urllib.parse.quote(query)
    
    # Modify the RSS URL to use the encoded query
    rss_url = f"https://news.google.com/rss/search?q={encoded_query}&hl=en-US&gl=US&ceid=US:en"
    
    # Parse the RSS feed
    feed = feedparser.parse(rss_url)
    
    articles = []
    
    # Define the date range for filtering articles (from January 2017 to June 2023)
    start_date = datetime(2017, 1, 1)
    end_date = datetime(2024, 9, 30)
    
    for entry in feed.entries:
        # Convert the published date to a datetime object
        published_date = datetime.strptime(entry.published, '%a, %d %b %Y %H:%M:%S %Z')
        
        # Check if the article is within the specified date range
        if start_date <= published_date <= end_date:
            # Extract the publisher/source (if available)
            publisher = entry.get('source', {}).get('title', 'Unknown Publisher')

            # Append relevant details to the articles list
            articles.append({
                'title': entry.title,
                'link': entry.link,
                'published': published_date,
                'publisher': publisher,
                'summary': entry.summary
            })
    
    return articles

# Expanded query to include more VHF-related keywords
search_query = (
    'Lassa fever OR Ebola OR viral hemorrhagic fever OR Marburg virus OR '
    'Crimean-Congo hemorrhagic fever OR Hantavirus OR Dengue OR Rift Valley fever OR '
    'Filovirus OR Arenavirus OR Hemorrhagic syndrome OR Severe fever with thrombocytopenia syndrome'
)

# Fetch articles
articles = fetch_google_news_rss(query=search_query)

# Check if we have articles
if articles:
    # Save to a DataFrame and export as CSV
    df = pd.DataFrame(articles)
    df.to_csv('google_news_rss_articles_filtered_expanded.csv', index=False)
    print(f"Fetched {len(df)} articles from Google News RSS.")
else:
    print("No articles found.")


Fetched 42 articles from Google News RSS.
